Load Library

In [ ]:
from keras.applications.xception import Xception
from keras.preprocessing import image
import numpy as np
from keras.layers import Dense, Dropout
from keras.models import Model
import os
from keras.preprocessing import image
from keras.utils.np_utils import to_categorical
from keras.metrics import categorical_accuracy
from keras.callbacks import Callback, ModelCheckpoint,LearningRateScheduler,EarlyStopping,ReduceLROnPlateau
from keras.optimizers import SGD,Adam

Load Driver

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Load data set

In [ ]:
##load data set for subject s with feature fea
##features to choose are feature_list=['MAV','WL','SSC','ZC','IEMG','VAR','WAMP','RMS','mDWT','HEMG','MAVS','ARC','MNF','PSR','SampEn','CC','HHT']
def Load_data(s,fea):
  if s<0 or s>27:
    print('subject not exist')
    return
  else:
    path='/content/drive/MyDrive/srtp-xsh/RGBimg_with_feature/s'+str(s)+'/'+str(fea)
    # s_weight='/content/drive/MyDrive/srtp-xsh/RGBimg_with_feature/s'+str(s)+'/'+str(fea)+'/weight.h5'
    train=path+'/train'
    test=path+'/test'
    X_train=[]
    Y_train=[]
    for x in os.listdir(train):

      f=train+'/'+x
      img=image.load_img(f)
      img=image.img_to_array(img)
      img=image.smart_resize(img, (100,200), interpolation='bilinear')
      label=x[3]
      X_train.append(img)
      Y_train.append(label)
    X_test=[]
    Y_test=[]
    for x in os.listdir(test):
      f=test+'/'+x
      img=image.load_img(f)
      img=image.img_to_array(img)
      img=image.smart_resize(img, (100,200), interpolation='bilinear')
      label=x[3]
      X_test.append(img)
      Y_test.append(label)
    X_train=np.asarray(X_train,np.float32)
    X_test=np.asarray(X_test,np.float32)
    Y_train=np.asarray(Y_train,np.float32)
    Y_test=np.asarray(Y_test,np.float32)

    num_example=X_train.shape[0]
    arr=np.arange(num_example)
    np.random.shuffle(arr)
    X_train=X_train[arr]
    Y_train=Y_train[arr]
    Y_train = to_categorical(Y_train,num_classes=52)
    Y_test = to_categorical(Y_test,num_classes=52)
    print('successfully load data for subject',str(s),'with feature',fea)
    print('shape of training data:',X_train.shape)
    print('shape of training label:',Y_train.shape)
    print('shape of test data:',X_test.shape)
    print('shape of test label:',Y_test.shape)
    return X_train,Y_train,X_test,Y_test,s_weight,path

In [ ]:
X_train,Y_train,X_test,Y_test,s_weight,path=Load_data(2,'RMS')

successfully load data for subject 2 with feature RMS
shape of training data: (364, 100, 200, 3)
shape of training label: (364, 52)
shape of test data: (156, 100, 200, 3)
shape of test label: (156, 52)


Test from here

In [ ]:
acc=[]
for tr in range(10):
  base_model=Xception(include_top=False, weights='imagenet', input_tensor=None, input_shape=(100,200,3), pooling='avg')
  x=base_model.output
  x = Dense(1024, activation='relu')(x)
  x = Dense(512, activation='relu')(x)
  predictions = Dense(52, activation='softmax')(x)

  model = Model(inputs=base_model.input, outputs=predictions)
  s_weight='/content/drive/MyDrive/srtp-xsh/RGBimg_with_feature/'+str(fea)+'/tr'+str(tr)'.h5'
  opt=Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,name='Adam')
  learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.5, min_lr=0.00001)
  model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['categorical_accuracy']) 
  callbacks = [ModelCheckpoint(s_weight, monitor='val_loss', save_best_only=True),learning_rate_reduction]
  history = model.fit(         X_train,
                      Y_train,
                      batch_size=32,
                      epochs=24,
                      validation_split=0.1,
                      callbacks=callbacks)
  model.load_weights(s_weight)
  loss, accuracy = model.evaluate(X_test,
                    Y_test)
  acc.append(accuracy)
  # print('loss = {0:.4f}, accuracy = {1:.4f}'.format(loss, accuracy))
    

In [ ]:
print(acc,np.array(acc).mean())

Load model

In [ ]:

base_model=Xception(include_top=False, weights='imagenet', input_tensor=None, input_shape=(100,200,3), pooling='avg')
x=base_model.output
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(52, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


overview of performance(skip this part if tuning is required)

In [ ]:
opt=Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,name='Adam')
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.5, min_lr=0.00001)
model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['categorical_accuracy']) 
callbacks = [ModelCheckpoint(s_weight, monitor='val_loss', save_best_only=True),learning_rate_reduction]
history = model.fit(         X_train,
                    Y_train,
                    batch_size=32,
                    epochs=24,
                    validation_split=0.1,
                    callbacks=callbacks)

Epoch 1/24
11/11 [==============================] - 14s 493ms/step - loss: 2.8542 - categorical_accuracy: 0.2367 - val_loss: 5.5958 - val_categorical_accuracy: 0.2973


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/24
11/11 [==============================] - 3s 228ms/step - loss: 1.0859 - categorical_accuracy: 0.7050 - val_loss: 6.0592 - val_categorical_accuracy: 0.1892
Epoch 3/24
11/11 [==============================] - 2s 226ms/step - loss: 0.6255 - categorical_accuracy: 0.8233 - val_loss: 5.8518 - val_categorical_accuracy: 0.2162
Epoch 4/24
11/11 [==============================] - 3s 228ms/step - loss: 0.5861 - categorical_accuracy: 0.8176 - val_loss: 17.7559 - val_categorical_accuracy: 0.1622

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 5/24
11/11 [==============================] - 3s 229ms/step - loss: 0.4065 - categorical_accuracy: 0.8601 - val_loss: 7.8930 - val_categorical_accuracy: 0.1892
Epoch 6/24
11/11 [==============================] - 3s 228ms/step - loss: 0.2871 - categorical_accuracy: 0.9188 - val_loss: 5.9344 - val_categorical_accuracy: 0.2973
Epoch 7/24
11/11 [==============================] - 3s 228ms/step - loss: 0.1439 - categ

In [ ]:
model.load_weights(s_weight)
loss, accuracy = model.evaluate(X_test,
                  Y_test)
print('loss = {0:.4f}, accuracy = {1:.4f}'.format(loss, accuracy))